In [85]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# LCMS Map Assemblage

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/lcms-training/blob/main/6-Map_Assemblage.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/lcms-training/blob/main/6-Map_Assemblage.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/redcastle-resources/lcms-training/blob/main/6-Map_Assemblage.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>


## Overview


This notebook teaches how to take raw model outputs and assemble final map classes


### Objective

In this tutorial, you learn how to manipulate raw model output GEE image arrays to create map outputs of a single class.

This tutorial uses the following Google Cloud services:

- `Google Earth Engine`

The steps performed include:

- Looking at raw GEE image array model output assets
- Manipulating the image arrays for a basic map assemblage
- Performing a more complicated map assemblage that balances omission and commission for Change

In [1]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
import geeViz.gee2Pandas as g2p
import inspect,operator
import matplotlib.pyplot as plt
import pandas as pd  
# from IPython.display import IFrame,display, HTML
ee = getImagesLib.ee
Map = getImagesLib.Map

# Can set the port used for viewing map outputs
Map.port = 1235
print('Done')


Initializing GEE
Successfully initialized
geeViz package folder: /opt/conda/lib/python3.10/site-packages/geeViz
PyTables is not installed. No support for HDF output.
Done


## Before you begin

### Set your current URL under `workbench_url`
* This will be in your URL/search bar at the top of the browser window you are currently in
* It will look something like `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/`

### Set a folder to use for all exports under `export_path_root` 
* It will be something like `projects/projectID/assets/someFolder`
* This folder does not have to already exist. If it does not exist, it will be created

In [9]:
workbench_url = 'https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com'
export_path_root  = 'projects/rcr-gee/assets/lcms-training'

print('Done')

Done


In [10]:
# Bring in all folders/collections that are needed
# These must already exist as they are created in previous notebooks
export_rawLCMSOutputs_collection = f'{export_path_root}/lcms-training_module-5_rawLCMSOutputs'

export_assembledLCMSOutputs_collection = f'{export_path_root}/lcms-training_module-6_assembledLCMSOutputs'


aml.create_asset(export_assembledLCMSOutputs_collection, asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(export_assembledLCMSOutputs_collection,writers = [],all_users_can_read = True,readers = [])

print('Done')

Found the following sub directories:  ['lcms-training', 'lcms-training_module-6_assembledLCMSOutputs']
Will attempt to create them if they do not exist
Asset projects/rcr-gee/assets/lcms-training already exists
Asset projects/rcr-gee/assets/lcms-training/lcms-training_module-6_assembledLCMSOutputs already exists
Updating permissions for:  projects/rcr-gee/assets/lcms-training/lcms-training_module-6_assembledLCMSOutputs
Done


In [75]:
Map.proxy_url = workbench_url
Map.clearMap()
# Bring in raw LCMS model outputs
raw_lcms = ee.ImageCollection(export_rawLCMSOutputs_collection)

# Bring in existing LCMS data for the class names, numbers, and colors
lcms_viz_dict = ee.ImageCollection("USFS/GTAC/LCMS/v2020-6").first().toDictionary().getInfo()
print(lcms_viz_dict)
    
# Get some info
products = raw_lcms.aggregate_histogram('product').keys().getInfo()


# Specify missing land cover codes in PRUSVI
# Talls Shrubs (2 and 6)  and snow/ice (13) are not present and need filled back in
# Change and Land_Use have all values, so just putting a very large number will cause it to skip it
missing_values = {'Change':[9999],
                  'Land_Cover':[2,6,13],
                  'Land_Use':[9999]
                 }
missing_names = {'Change':[],
                  'Land_Cover':['Tall Shrubs & Trees Mix (SEAK Only)',
                                'Tall Shrubs (SEAK Only)',
                                'Snow or Ice'],
                  'Land_Use':[]
                 }
def arrayToImage(img,bandNames):
    return img.arrayProject([0])\
            .arrayFlatten([bandNames])\
            .copyProperties(img,['system:time_start'])

# Function to find the most probable class
def getMostProbableClass(raw_lcms_product_yr,product):
    # Pull the index of the most probable value
    # Since 0 is not used for LCMS outputs, add 1
    max_prob_class = raw_lcms_product_yr.arrayArgmax().arrayGet(0).add(1).byte().rename([product])
    max_prob_class = ee.Image(max_prob_class)

    # Handle missing land cover types
    # Since the index is used, missing classes will be coded wrong without this step
    # For PRUSVI, only land cover is missing classes
    for missing_value in missing_values[product]:
        max_prob_class = max_prob_class.where(max_prob_class.gte(missing_value),max_prob_class.add(1))
    
    null_code = lcms_viz_dict[f'{product}_class_values'][-1]
    max_prob_class = max_prob_class.unmask(null_code)
    return max_prob_class.copyProperties(raw_lcms_product_yr,['system:time_start'])

# Iterate across each product and assemble the most probable class collection from the raw
for product in products:
    # Pull the class names
    class_names = lcms_viz_dict[f'{product}_class_names'][:-1]
    class_names = [nm for nm in class_names if nm not in missing_names[product]]
    
    product_title = product.replace('_',' ')
    
    # Filter to only include the product of interest
    raw_lcms_product = raw_lcms.filter(ee.Filter.eq('product',product))
    
    # Find the max probability class
    maxProb_lcms_product = raw_lcms_product.map(lambda raw_lcms_product_yr:getMostProbableClass(raw_lcms_product_yr,product).set(lcms_viz_dict))
    
    # Convert raw model probabilities to bands for exploration
    raw_lcms_product = raw_lcms_product.map(lambda img:arrayToImage(img,class_names))
    
    # Add the raw model probabilities to query
    Map.addTimeLapse(raw_lcms_product.limit(3),{},f'Raw LCMS {product_title}')
    
    # Add the assembed final class maps
    Map.addTimeLapse(maxProb_lcms_product.limit(3),{'autoViz':True},f'Most Probable LCMS {product_title}')
Map.turnOnInspector()
Map.view()
print('Done')

{'Change_class_names': ['Stable', 'Slow Loss', 'Fast Loss', 'Gain', 'Non-Processing Area Mask'], 'Change_class_palette': ['3d4551', 'f39268', 'd54309', '00a398', '1b1716'], 'Change_class_values': [1, 2, 3, 4, 5], 'Land_Cover_class_names': ['Trees', 'Tall Shrubs & Trees Mix (SEAK Only)', 'Shrubs & Trees Mix', 'Grass/Forb/Herb & Trees Mix', 'Barren & Trees Mix', 'Tall Shrubs (SEAK Only)', 'Shrubs', 'Grass/Forb/Herb & Shrubs Mix', 'Barren & Shrubs Mix', 'Grass/Forb/Herb', 'Barren & Grass/Forb/Herb Mix', 'Barren or Impervious', 'Snow or Ice', 'Water', 'Non-Processing Area Mask'], 'Land_Cover_class_palette': ['005e00', '008000', '00cc00', 'b3ff1a', '99ff99', 'b30088', 'e68a00', 'ffad33', 'ffe0b3', 'ffff00', 'aa7700', 'd3bf9b', 'ffffff', '4780f3', '1b1716'], 'Land_Cover_class_values': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 'Land_Use_class_names': ['Agriculture', 'Developed', 'Forest', 'Non-Forest Wetland', 'Other', 'Rangeland or Pasture', 'Non-Processing Area Mask'], 'Land_Use_

Done


127.0.0.1 - - [18/Aug/2023 19:32:54] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


In [ ]:
# While this assemblage method works well, you can introduce various rules to talor the final 
# map product to different audiences
# In the case of LCMS, there is a focus on forest applications
# We know that our models are generally not very confident about classifying change largely
# due to the limited number of training samples that experience change
# We also know our land use classes are not mutually exclusive (e.g. non-forest wetlands can potentially occur in any land use)
# Therefore we have a lot of confusion in our land use outputs


In [32]:
Map.clearMap()
# Now lets export assembled assets
# Optionally, we can export using the tile grid approach
# For PRUSVI, LCMS does not need to export using this approach, but this is how you would set it up
# First, we'll set up the study area and a tile to export across

studyArea = ee.FeatureCollection('projects/lcms-292214/assets/R8/PR_USVI/Ancillary/prusvi_boundary_buff2mile')

# Set the size (in meters) of the tiles
# We can likely use a large tile for this step
# If exports fail, reducing the tileSize is likely to help
tileSize = 240000


# Set the projection
crs = getImagesLib.common_projections['NLCD_CONUS']['crs']
transform  = getImagesLib.common_projections['NLCD_CONUS']['transform']
scale = None
projection = ee.Projection(crs,transform)

# Set up years to apply models across
apply_years = list(range(1985,2022+1))

# Get the grid
grid = studyArea.geometry().coveringGrid(projection.atScale(tileSize))
Map.addLayer(grid,{},'Tile Grid {}m'.format(tileSize))

Map.centerObject(grid)
Map.view()

Adding layer: Tile Grid 240000m
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


127.0.0.1 - - [18/Aug/2023 18:49:22] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


* Iterate across each year and assemble full stack output
* [Example of fully assembed bands](https://developers.google.com/earth-engine/datasets/catalog/USFS_GTAC_LCMS_v2022-8#bands)
* We will not be producing the qa-bits bands

In [67]:
# Iterate across each year, assemble full stack output (
# products = ['Land_Cover','Land_Use']
study_area_name = 'PRUSVI'
version = '2022-Training'

for apply_year in apply_years[:1]:
    assembled_list = []
    probability_list = []
    for product in products:
        # Pull the class names
        class_names = lcms_viz_dict[f'{product}_class_names'][:-1]
        
        # Get rid of missing class names
        class_names = [nm for nm in class_names if nm not in missing_names[product]]
        
        # Get rid of any characters that aren't allowed
        class_names = [nm.replace(' ','-').replace('/','-').replace('&','and') for nm in class_names]
        
        probability_names = [f'{product}_Raw_Probability_{class_name}' for class_name in class_names]
        
        product_title = product.replace('_',' ')

        # Filter to only include the product of interest
        raw_lcms_product_yr = raw_lcms\
                            .filter(ee.Filter.calendarRange(apply_year,apply_year,'year'))\
                            .filter(ee.Filter.eq('product',product)).first()

        # Find the max probability class
        maxProb_lcms_product_yr = getMostProbableClass(raw_lcms_product_yr,product)
        assembled_list.append(maxProb_lcms_product_yr)
        
        # Convert raw model probabilities to bands for exploration
        prob_lcms_product_yr = arrayToImage(raw_lcms_product_yr,probability_names)
        probability_list.append(prob_lcms_product_yr)
        
    final_stack = ee.Image.cat([ee.Image.cat(assembled_list),ee.Image.cat(probability_list)])\
                    .copyProperties(raw_lcms_product_yr,['study_area','year'])

    exportName = f'LCMS_{study_area_name}_v{version}_{apply_year}'
    exportPath = f'{export_rawLCMSOutputs_collection}/{exportName}'

    print(exportPath)

    # getImagesLib.exportToAssetWrapper(predicted,exportName,exportPath,{'.default':'sample'},studyArea,scale,crs,transform,overwrite=False)


print('Done')

projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_PRUSVI_v2022-Training_1985
Done


In [ ]:
# Can track tasks here or at https://code.earthengine.google.com/tasks
# If you'd like to track the tasks, use this:
# tml.trackTasks2()

# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the collection of all images
# aml.batchDelete(export_rawLCMSOutputs_collection, type = 'imageCollection')

print('done')